# Домашнее задание №7: ДЗ7-бонусное

##  Содержание <a name = 'outline'></a>

* [Проведите анализ чувствительности, проверьте сходимость, измерьте тайминги, 25 баллов](#task_1)
* [Ускорьте вычисления Python с использованием любой из имеющихся возможностей (PyBind11, ctypes, cython, numba), 25 баллов](#task_2)
* [Попробуйте добавить параллелизм в вычисления, 25 баллов](#task_3)
* Оформите и представьте результаты, 25 баллов

Зафиксируйте какую-либо многомерную скалярную функцию и реализуйте для неё анализ чувствительности по методу Соболя на Python с использованием библиотеки SALib.

In [1]:
import time
import numpy as np
from numba import njit, prange
from SALib.sample import saltelli
from SALib.analyze import sobol

#### Проведите анализ чувствительности, проверьте сходимость, измерьте тайминги, 25 баллов  <a name = "task_1"/>

In [8]:
a = 2
b = 0.7

In [9]:
def evaluate_model(x):
    return a * np.sin(x[2]) ** 2 * np.cos(x[2]) + b * np.sin(x[0])**3 * np.cos(x[1])**3

In [10]:
problem = {
    'num_vars' : 3,
    'names' : ['x1', 'x2', 'x3'],
    'bounds' : [[-6, 6],
                [-6, 6],
                [-6, 6]
                ]
}

In [11]:
n = 100000
start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))
Y = np.zeros([param_values.shape[0]])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = evaluate_model(X)

print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
Si = sobol.analyze(problem, Y)

print("SA took %s seconds" %(time.time() - start))
print(Si['S1'])
print(Si['ST'])
print("x1-x2:", Si['S2'][0,1])
print("x1-x3:", Si['S2'][0,2])
print("x2-x3:", Si['S2'][1,2])

samples generation took 5.383628845214844 seconds
model evaluation took 6.311183214187622 seconds
SA took 6.0586020946502686 seconds
[ 1.06024034e-03 -4.35544182e-04  8.51405649e-01]
[0.14813827 0.14709553 0.85139607]
x1-x2: 0.14726423578047473
x1-x3: -5.630460967198658e-05
x2-x3: 0.0004021976428819629


#### Ускорьте вычисления Python с использованием любой из имеющихся возможностей (PyBind11, ctypes, cython, numba), 25 баллов  <a name = "task_2"/>

In [12]:
@njit
def opt_evaluate_model(x):
    return a * np.sin(x[2]) ** 2 * np.cos(x[2]) + b * np.sin(x[0])**3 * np.cos(x[1])**3

In [13]:
n = 100000
start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))
Y = np.zeros([param_values.shape[0]])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = opt_evaluate_model(X)

print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
Si = sobol.analyze(problem, Y)

print("SA took %s seconds" %(time.time() - start))

samples generation took 5.350808143615723 seconds
model evaluation took 0.6698927879333496 seconds
SA took 5.232341051101685 seconds


#### Попробуйте добавить параллелизм в вычисления, 25 баллов  <a name = "task_3"/>

In [15]:
n = 100000
start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))
Y = np.zeros([param_values.shape[0]])

start = time.time()
for i in prange(100000):
    Y[i] = opt_evaluate_model(param_values[i])

print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
Si = sobol.analyze(problem, Y)

print("SA took %s seconds" %(time.time() - start))

/Users/ponomarevandrew/opt/anaconda3/lib/python3.8/site-packages/SALib/sample/saltelli.py:94: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (100000) is equal to `2^n`.
        
  warnings.warn(msg)


samples generation took 5.732519865036011 seconds
model evaluation took 0.05318713188171387 seconds
SA took 5.874913692474365 seconds
